In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


In [2]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

In [3]:
def get_reviews(product_list, index_start=0, index_end=316):
    review_df = pd.DataFrame(columns=['product','user_rating', 'title', 'user_review'])
    idx = 0
    for index in range(index_start, index_end + 1):
        try:
            product_name = product_list['Name'][index]
            product_link = product_list['Link'][index]
            pages = product_list['Pages'][index]
            
            print('Index : Page / Pages  :  Product Name')
            for page in range(pages):
                print(index, ':', page +
                    1, '/', pages, ' : ', product_name)

                replace_string = 'product-reviews'
                review_link = product_link[0: product_link.index(
                    'FLIPKART') + 8]
                review_link = review_link.split('/')
                review_link[4] = replace_string
                review_link = '/'.join(review_link)

                url = review_link + '&page=' + str(page+1)
                driver.get(url)
                
                try:
                    WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located(
                            (By.XPATH, "//span[text()='READ MORE']")))

                    element = driver.find_elements(
                        By.XPATH, "//span[text()='READ MORE']")  # For expanding reviews

                    for i in element:
                        if i.text == 'READ MORE':
                            i.click()
                except:
                    pass
                
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                reviews = soup.find_all(
                    'div', attrs={'class': 'col _2wzgFH K0kLPL'})
                for review in reviews:
                    row = review.find(
                        'div', attrs={'class': 'row'})
                    user_rating = row.find('div').text
                    title = row.find('p').text
                    user_review = review.find('div', attrs={'class': 't-ZTKy'}).text.replace('READ MORE', '')
                    
                    res = [product_name, user_rating,
                        title, user_review]
                    review_df.loc[idx] = res
                    idx += 1
        except:
            break        

        print()
    return review_df

In [4]:
product_list = pd.read_csv('unique_product_list_with_ratings.csv')
q = product_list['Pages'].quantile(.95)
product_list = product_list[product_list['Pages'] < q]
product_list.reset_index(drop=True, inplace=True)
product_list['Pages'] = product_list['Pages'].astype(dtype='int')
product_list['Total Reviews'] = product_list['Total Reviews'].apply(lambda x: x.replace(',', '')).astype(
    dtype='int')
print(len(product_list))
product_list.head(1)


317


,Brand,Name,Price,Link,Features,Overall Rating,Total Reviews,Pages,5,4,3,2,1
0,Asus Mobile,ASUS ZenFone 5Z,"36,299",https://www.flipkart.com/asus-zenfone-5z-midni...,['6 GB RAM | 128 GB ROM | Expandable Upto 2 TB...,4.5,4616,462,15005,4566,979,339,1052


In [5]:
product_list['Total Reviews'][1]

131

Fetching data in chuncks

In [7]:
# idx = [-1, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 316]
idx = [-1, 25, 50, 75, 100]
idx = [0, 2]

if True:
    INDEX_START = 10
    INDEX_END = 11

for i in range(len(idx) - 1):    
    INDEX_START = idx[i] + 1
    INDEX_END = idx[i + 1]
    
    try:
        review_df = get_reviews(product_list=product_list, index_start=INDEX_START, index_end=INDEX_END)
    except:
        review_df.to_csv(
            f'reviews_{INDEX_START}_to_{INDEX_END}.csv', index=False)
        break
    review_df.to_csv(f'reviews_{INDEX_START}_to_{INDEX_END}.csv', index=False)
    


Index : Page / Pages  :  Product Name
1 : 1 / 14  :  ASUS ROG 5s
1 : 2 / 14  :  ASUS ROG 5s
1 : 3 / 14  :  ASUS ROG 5s
1 : 4 / 14  :  ASUS ROG 5s
1 : 5 / 14  :  ASUS ROG 5s
1 : 6 / 14  :  ASUS ROG 5s
1 : 7 / 14  :  ASUS ROG 5s
1 : 8 / 14  :  ASUS ROG 5s
1 : 9 / 14  :  ASUS ROG 5s
1 : 10 / 14  :  ASUS ROG 5s
1 : 11 / 14  :  ASUS ROG 5s
1 : 12 / 14  :  ASUS ROG 5s
1 : 13 / 14  :  ASUS ROG 5s
1 : 14 / 14  :  ASUS ROG 5s

Index : Page / Pages  :  Product Name
2 : 1 / 116  :  ASUS 6Z
2 : 2 / 116  :  ASUS 6Z
2 : 3 / 116  :  ASUS 6Z
2 : 4 / 116  :  ASUS 6Z


Printing number of reviews for each chunk

In [19]:
# idx = [-1, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 316]
print('Total Reviews:', product_list['Total Reviews'].sum())

if True:
    INDEX_START = 10
    INDEX_END = 11

for i in range(len(idx) - 1):
    INDEX_START = idx[i] + 1
    INDEX_END = idx[i + 1]
    
    reviews = product_list['Total Reviews'].iloc[INDEX_START: INDEX_END + 1].sum()
    print(f'Total Reviews from {INDEX_START} to {INDEX_END}:', reviews)


Total Reviews: 672580
Total Reviews from 10 to 11: 108


Printing lenght of fetched data (from files)

In [ ]:
# idx = [-1, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 316]

reviews = 0
if True:
    INDEX_START = 10
    INDEX_END = 11
    
for i in range(len(idx) - 1):
    INDEX_START = idx[i] + 1
    INDEX_END = idx[i + 1]

    file = pd.read_csv(f'reviews_{INDEX_START}_to_{INDEX_END}.csv')
    reviews += len(file)
    print(f'reviews_{INDEX_START}_to_{INDEX_END}:', len(file))
print(reviews)

Fetching the Overall Ratings and Reviews

In [ ]:
product_list = pd.read_csv('unique_product_list.csv')
product_list.reset_index(drop=True, inplace=True)
product_list = pd.concat([product_list, pd.DataFrame(
    columns=['Overall Rating', 'Total Reviews', 'Pages', '5', '4', '3', '2', '1'])], axis=1)

driver = webdriver.Chrome(options=options)
for index in range(len(product_list)):
    try:
    # if True:
        product_name = product_list['Name'].iloc[index]
        product_link = product_list['Link'].iloc[index]

        replace_string = 'product-reviews'
        review_link = product_link[0: product_link.index('FLIPKART') + 8]
        review_link = review_link.split('/')
        review_link[4] = replace_string
        review_link = '/'.join(review_link)

        driver.get(review_link)
        time.sleep(1)
        
        print()
        print(index, '- Product:', product_name)
        print('Link:', review_link)
        pages = 1

        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        try:
            pages = soup.find('div', attrs={'class':'_2MImiq _1Qnn1K'}).find('span').text
            pages = pages.split()
            pages = int(pages[len(pages) - 1].replace(',', ''))  # String
            print('Pages:', pages)
        except:
            print('Sigle page review')
            
        overall_rating = soup.find(class_='_2d4LTz')
        overall_rating = float(overall_rating.text)  # Float
        
        total_reviews = soup.find_all('div', attrs={'class': 'row _2afbiS'})[1].text
        total_reviews = total_reviews[ : total_reviews.index('Reviews') - 1]

        review_ratings, rating_count = soup.find_all(
            class_='_26f_zl'), soup.find_all(class_='_1uJVNT')  # List
        for i in range(len(rating_count)):
            review_ratings[i] = int(review_ratings[i].text)
            rating_count[i] = int(rating_count[i].text.replace(',', ''))

        ratings_star_counts = dict(
            zip(review_ratings, rating_count))  # dictionary

        product_list['Overall Rating'].iloc[index] = overall_rating
        product_list['Total Reviews'].iloc[index] = total_reviews
        product_list['Pages'].iloc[index] = pages
        product_list['5'].iloc[index] = ratings_star_counts[5]
        product_list['4'].iloc[index] = ratings_star_counts[4]
        product_list['3'].iloc[index] = ratings_star_counts[3]
        product_list['2'].iloc[index] = ratings_star_counts[2]
        product_list['1'].iloc[index] = ratings_star_counts[1]
    except:
        continue

product_list.to_csv('unique_product_list_with_ratings.csv', index=False)
